<a href="https://colab.research.google.com/github/kittimaxz/Project_BoneAge/blob/main/Test_BoNet_RSNA_RHPE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/BCV-Uniandes/Bonet.git
!cd Bonet

Cloning into 'Bonet'...
remote: Enumerating objects: 52, done.
remote: Total 52 (delta 0), reused 0 (delta 0), pack-reused 52
Unpacking objects: 100% (52/52), done.


In [2]:
import numpy as np # linear algebra # เป็นการ import numpy เพื่อเปิดใช้ฟังก์ชันในการทำงาน
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv) # เป็น library ในการจัดการ dataframe

In [3]:
import Bonet

In [4]:
# Standard lib imports
import os
import csv
import glob
import time
import argparse
import warnings
import pandas as pd
import os.path as osp

In [5]:
# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
!pip install horovod
import horovod.torch as hvd
from torchvision import transforms
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.5 MB 5.7 MB/s 
  Created wheel for horovod: filename=horovod-0.26.1-cp38-cp38-linux_x86_64.whl size=28368570 sha256=2379dbbea2e3c02a9c0af70ae1285e90095829d0b360a2189d714c3b07949f87
  Stored in directory: /root/.cache/pip/wheels/eb/0b/90/d53058f75f3ae3db9557f3e55dd8c016b2397e9b38557c8b66
Successfully built horovod


In [6]:
# Other imports
from tqdm import tqdm
import pdb

In [7]:
warnings.filterwarnings("ignore")

In [8]:
parser = argparse.ArgumentParser()

In [9]:
from google.colab import drive # เชื่อม drive ของเรา ถ้าเชื่อมสำเร็จจะขึ้นคำว่าMounted at /content/drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import os # os จัดการเกี่ยวกับไฟล์ต่างๆ ในโฟล์เดอร์

In [11]:
path = '/content/drive/My Drive/Model_Bonet'  # ชี้ว่า data เราอยู่โฟล์เดอร์ไหน

In [12]:
ls

Bonet/  drive/  sample_data/


In [13]:
# Dataloading-related settings
parser.add_argument('--heatmaps', default=False, action='store_true',
                help='Train model with gaussian heatmaps')
parser.add_argument('--cropped', default=False, action='store_true',
                help='Train model with cropped images according to bbox')
parser.add_argument('--dataset', default='RSNA', type=str,choices=['RSNA','RHPE'],
                help='Dataset to perform training')

parser.add_argument('Data/Train/', default='Data/Train/', type=str,
                help='path to train data folder')
parser.add_argument('Data/Train/', default='train.csv', type=str,
                help='path to BAA annotations file')
parser.add_argument('Data/', default='train.json',
                type=str, help='path to ROIs annotations in coco format')

parser.add_argument('Data/Validation/', default='Data/Validation/', type=str,
                help='path to val data folder')
parser.add_argument('Data/Validation/', default='validation.csv', type=str,
                help='path to BAA annotations file')
parser.add_argument('Data/', default='validation.json',
                type=str, help='path to ROIs annotations in coco format')

parser.add_argument('--save-folder', default='TRAIN/new_test/',
                help='location to save checkpoint models')
parser.add_argument('--snapshot', default='boneage_bonet_weights.pth',
                help='path to weight snapshot file')
parser.add_argument('--optim-snapshot', type=str,
                default='boneage_bonet_optim.pth',
                help='path to optimizer state snapshot')

parser.add_argument('--eval-first', default=False, action='store_true',
                help='evaluate model weights before training')
parser.add_argument('-j', '--workers', default=4, type=int, metavar='N',
                help='number of data loading workers (default: 4)')

_StoreAction(option_strings=['-j', '--workers'], dest='workers', nargs=None, const=None, default=4, type=<class 'int'>, choices=None, help='number of data loading workers (default: 4)', metavar='N')

In [14]:
# Training procedure settings
parser.add_argument('--batch-size', default=1, type=int,
                help='Batch size for training')
parser.add_argument('--epochs', type=int, default=20,
                help='upper epoch limit')
parser.add_argument('--lr', '--learning-rate', default=1e-5, type=float,
                help='initial learning rate')
parser.add_argument('--patience', default=2, type=int,
                help='patience epochs for LR decreasing')
parser.add_argument('--start-epoch', type=int, default=1,
                help='epoch number to resume')
parser.add_argument('--seed', type=int, default=1111,
                    help='random seed')
parser.add_argument('--log-interval', type=int, default=30, metavar='N',
                    help='report interval')

parser.add_argument('--gpu', type=str, default='2,3')

_StoreAction(option_strings=['--gpu'], dest='gpu', nargs=None, const=None, default='2,3', type=<class 'str'>, choices=None, help=None, metavar=None)

In [15]:
args = parser.parse_args(args=[])
args

usage: ipykernel_launcher.py [-h] [--heatmaps] [--cropped]
                             [--dataset {RSNA,RHPE}]
                             [--save-folder SAVE_FOLDER] [--snapshot SNAPSHOT]
                             [--optim-snapshot OPTIM_SNAPSHOT] [--eval-first]
                             [-j N] [--batch-size BATCH_SIZE]
                             [--epochs EPOCHS] [--lr LR] [--patience PATIENCE]
                             [--start-epoch START_EPOCH] [--seed SEED]
                             [--log-interval N] [--gpu GPU]
                             Data/Train/ Data/Train/ Data/ Data/Validation/
                             Data/Validation/ Data/
ipykernel_launcher.py: error: the following arguments are required: Data/Train/, Data/Train/, Data/, Data/Validation/, Data/Validation/, Data/


SystemExit: ignored

In [ ]:
args_dict = vars(args)
print('Argument list to program')
print('\n'.join(['--{0} {1}'.format(arg, args_dict[arg])
                 for arg in args_dict]))
print('\n\n')

In [ ]:
torch.manual_seed(args.seed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu

In [ ]:
if not os.path.exists(args.save_folder):
    os.makedirs(args.save_folder)

In [ ]:
# Horovod settings
hvd.init()
torch.cuda.set_device(hvd.local_rank())
torch.cuda.manual_seed(hvd.size())

args.distributed = hvd.size() > 1
args.rank = hvd.rank()
args.size = hvd.size()

In [ ]:
# CREATE THE NETWORK ARCHITECTURE AND LOAD THE BEST MODEL
if args.heatmaps:
    from Bonet.models.bonet_heatmap import BoNet
else:
    from Bonet.models.bonet import BoNet

In [ ]:
net = BoNet()

In [ ]:
if args.rank == 0:
    print('---> Number of params: {}'.format(
        sum([p.data.nelement() for p in net.parameters()])))

In [ ]:
if osp.exists(args.snapshot):
    model_to_load=args.snapshot
else:
    model_to_load=args.save_folder+'/'+args.snapshot

In [ ]:
if osp.exists(model_to_load) and args.rank == 0:
    print('Loading state dict from: {0}'.format(model_to_load))
    snapshot_dict = torch.load(model_to_load, map_location=lambda storage, loc: storage)
    weights= net.state_dict()
    new_snapshot_dict=snapshot_dict.copy()
    for key in snapshot_dict:
        if key not in weights.keys():
            new_key='inception_v3.'+key
            new_snapshot_dict[new_key]=snapshot_dict[key]
            new_snapshot_dict.pop(key)

    net.load_state_dict(new_snapshot_dict)

net = net.to(device)

In [ ]:
# Criterion
criterion = nn.L1Loss()

In [ ]:
# Optimizer
optimizer = optim.Adam(net.parameters(), lr=args.lr * args.size)
annealing = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, factor=0.8, patience=args.patience, cooldown=5,
    min_lr=0.00001, eps=0.00001, verbose=True)

In [ ]:
if osp.exists(args.optim_snapshot):
    optim_to_load=args.optim_snapshot
else:
    optim_to_load=args.save_folder+'/'+args.optim_snapshot

In [ ]:
if osp.exists(optim_to_load):
    print('loading optim snapshot from {}'.format(optim_to_load))
    optimizer.load_state_dict(torch.load(optim_to_load, map_location=lambda storage,
                                             loc: storage))

In [ ]:
# Horovod
hvd.broadcast_parameters(net.state_dict(), root_rank=0)

optimizer = hvd.DistributedOptimizer(
    optimizer, named_parameters=net.named_parameters())
hvd.broadcast_optimizer_state(optimizer, root_rank=0)
group = optimizer.param_groups[0]
group['betas'] = (float(group['betas'][0]), float(group['betas'][1]))

In [ ]:
# Dataloaders
train_transform = transforms.Compose([transforms.Resize((500, 500)),
                               transforms.RandomAffine(
                                   20, translate=(0.2, 0.2),
                                   scale=(1, 1.2)),
                               transforms.RandomHorizontalFlip(),
                               transforms.ToTensor()])
train_transform

In [ ]:
val_transform = transforms.Compose([transforms.Resize((500, 500)),
                               transforms.ToTensor()])
val_transform

In [ ]:
if args.heatmaps:
    from Bonet.data.data_loader import Boneage_HeatmapDataset as Dataset
else:
    from Bonet.data.data_loader import BoneageDataset as Dataset

In [ ]:
train_dataset = Dataset(args.data_train, args.ann_path_train,args.rois_path_train,
                                   img_transform=train_transform,crop=args.cropped,dataset=args.dataset)
val_dataset = Dataset(args.data_val, args.ann_path_val,args.rois_path_val,
                                 img_transform=val_transform,crop=args.cropped,dataset=args.dataset)